In [2]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from NoEmbedRNN import OpSel
from NoEmbedRNN import MemSel
from NoEmbedRNN import RNN as oldRNN
from NoEmbedRNN import MemRNN  as oldMemRNN
from NoEmbedRNN import HistoryRNN
from rl_rnn import RLRNN
from rl_rnn_mem import RLRNNMEM
from ops import Operations
from session import *
from data_gen import *
import pickle
from functools import reduce

#modelName = 'RLRNN'
modelName = 'RNN'
old = False
func = 'np_add-5ops'
name = 'state_size_100-num_samples_1500-batch_size_100-test_ratio_0.33-learning_rate_0.005-epsilon_0.001-num_features_4-norm_True-softmax_sat_100-state_fn_relu-smax_pen_r_0.0-total_num_epochs_40000-augument_grad_True-relaunch_True-seed_52444'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
ops.ops = cfg["used_ops_obj"]
ops.num_of_ops = len(ops.ops)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        if old: mem = oldMemRNN(cfg, ops)
        else:   mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        if old: model = oldRNN(cfg, ops, mem)
        else:   model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "RLRNN":
        ops_env = cfg["used_ops_env"]
        mem = RLRNNMEM(cfg, ops_env) 
        model = RLRNN(cfg, ops_env, mem) 
        res = restore_selection_RL_RNN(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
timestep 0
timestep 1
timestep 2
timestep 3
timestep 4
[<tf.Variable 'W_mem:0' shape=(104, 100) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b_mem:0' shape=(100,) dtype=float64_ref>, <tf.Variable 'W2_mem:0' shape=(100, 4) dtype=float64_ref>, <tf.Variable 'RNN_mem/Params/b2_mem:0' shape=(4,) dtype=float64_ref>, <tf.Variable 'W:0' shape=(104, 100) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b:0' shape=(100,) dtype=float64_ref>, <tf.Variable 'W2:0' shape=(100, 6) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b2:0' shape=(6,) dtype=float64_ref>, <tf.Variable 'W3:0' shape=(6, 4) dtype=float64_ref>, <tf.Variable 'RNN_op/Params/b3:0' shape=(4,) dtype=float64_ref>]
norming the grads
grads are
[(<tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/clip_by_global_norm/_0:0' shape=(104, 100) dtype=float64>, <tf.Variable 'W_mem:0' shape=(104, 100) dtype=float64_ref>), (<tf.Tensor 'RNN_op/Grads/clip_by_global_norm/RNN_op/Grads/cl

In [ ]:
def print_errors(error_lst, name):
    print("")
    print("Total", reduce((lambda x, y: x + y), error_lst))
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [ ]:
def print_matrix(total_error, matrix_lst, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
    print("")

In [ ]:
def print_ops_matrix(total_error, matrix_lst, ops_list, indeces = None):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        if indeces is not None and elem not in indeces: continue 
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            if len(matrix[elem]) < 2:
                index = matrix[elem][0]
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
    print("")

In [ ]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")

#print_errors(res["train_math_error"], "train_math_error")
#print_errors(res["test_math_error"], "test_math_error")

In [ ]:
#Softmax Hardmax Print
print_matrix(res["total_loss_traind_train"][0], res["softmaxes_traind_train"][0])
print_ops_matrix(res["total_loss_traind_train"][0], res["softmaxes_traind_train"][0], cfg["used_ops_obj"])

##RL print - train OP selections
#print_matrix(res["train_math_error"][0], res["train_selections"][0])
#print_ops_matrix(res["train_math_error"][0], res["train_selections"][0], cfg["used_ops_env"].ops)

In [ ]:
#Softmax Hardmax Print

#print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])
#print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

##RL print - train MEM selections
print_matrix(res["train_math_error"][0], res["train_selections_mem"][0])
print_ops_matrix(res["train_math_error"][0], res["train_selections_mem"][0], cfg["used_ops_env"].ops_mem)

In [ ]:
##RL print - test OP selections
print_matrix(res["test_math_error"][0], res["test_selections"][0])
print_ops_matrix(res["test_math_error"][0], res["test_selections"][0], cfg["used_ops_env"].ops)

In [ ]:
##RL print - test MEM selections
print_matrix(res["test_math_error"][0], res["test_selections_mem"][0])
print_ops_matrix(res["test_math_error"][0], res["test_selections_mem"][0], cfg["used_ops_env"].ops_mem)

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_train"][0])

In [ ]:
wrong_res = np.nonzero(np.round(np.apply_along_axis(np_add, 1, res["outputs_testd_train"][0][4] - res["batchesY_test"][0]), 2))[0]
wrong_res 

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"], wrong_res)

In [ ]:
#print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
print_ops_matrix(res["total_loss_testd_test"][0], res["softmaxes_testd_test_mem"][0], cfg["used_ops_obj"])

In [ ]:
print_matrix(res["total_loss_testd_test"][0], res["outputs_testd_test_mem"][0])

In [ ]:
res['last_hardmax_state_train_mem']

In [ ]:
cfg['num_features']

In [3]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
#res_pred = predict_form_sess(model, cfg, x_test[0:100,], res['last_hardmax_state_train'],res['last_hardmax_state_train_mem'], path = model_path, mode="hard")

In [7]:
x[0:1]

array([[  9.09403021, -65.62693699,  70.5863619 , -63.95531784]])

In [ ]:
x_test[0:100,]

In [ ]:
res_pred

In [ ]:
len(res["softmaxes_traind_train"])